In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as MAE
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_selection import SelectKBest

In [21]:
home_data = pd.read_csv("train.csv")
#home_data = home_data.dropna(axis='columns').head()
y = home_data.SalePrice
home_data.columns


Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [45]:
#Dropping Nan Valued Columns
#home_data = home_data.dropna(axis='columns').head()
#home_data.info()

home_data.loc[:, 'Neighborhood'].unique()
home_data.isna().sum()

Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
                 ... 
BedroomAbvGr        0
KitchenAbvGr        0
KitchenQual         0
TotRmsAbvGrd        0
Functional          0
Fireplaces          0
FireplaceQu       690
GarageType         81
GarageYrBlt        81
GarageFinish       81
GarageCars          0
GarageArea          0
GarageQual         81
GarageCond         81
PavedDrive

In [23]:
#Column Names
home_data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [47]:
cols = home_data.columns
num_cols = home_data._get_numeric_data().columns

features = ['LotArea', 'BsmtFullBath', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'GarageCars', 'GarageArea', 'YrSold', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'MoSold']
X = home_data[features]
print(X.shape)




(1460, 12)


LotArea         0
BsmtFullBath    0
YearBuilt       0
1stFlrSF        0
2ndFlrSF        0
GarageCars      0
GarageArea      0
YrSold          0
FullBath        0
BedroomAbvGr    0
TotRmsAbvGrd    0
MoSold          0
dtype: int64

In [48]:
from sklearn.preprocessing import StandardScaler

features = ['LotArea','MiscVal','2ndFlrSF','PoolArea','BsmtFinSF2','GrLivArea','TotalBsmtSF', 'GarageCars']
X = home_data[features]
X.isna().sum()

LotArea        0
MiscVal        0
2ndFlrSF       0
PoolArea       0
BsmtFinSF2     0
GrLivArea      0
TotalBsmtSF    0
GarageCars     0
dtype: int64

In [35]:
sc = StandardScaler()
sc.fit(X)
X = sc.transform(X)

In [36]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)


In [37]:
#Getting the best max_leaf_nodes
def get_accuracy(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = RandomForestRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    acc = MAE(val_y, preds_val)
    
    return acc

    
max_leaf = [x for x in range(50, 250)]

 
best_accuracy = {}  
for i in range(len(max_leaf)):
    best_accuracy[i] = get_accuracy(max_leaf[i], train_X,val_X, train_y, val_y)

In [39]:
#Getting best n_estimators
def get_accuracy(n_estimators, train_X, val_X, train_y, val_y):
    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=4, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    acc = MAE(val_y, preds_val)
    return acc

best_estimator = [x for x in range(50, 250)]
 
best_accuracy_n_estimator = {}  
for i in range(len(max_leaf)):
    best_accuracy_n_estimator[i] = get_accuracy(best_estimator[i], train_X,val_X, train_y, val_y)

In [38]:
#Best max_leaf
min_key = min(best_accuracy, key=best_accuracy.get)
print(min_key)

4


In [40]:
#Best n_estimator
min_key = min(best_accuracy_n_estimator, key=best_accuracy_n_estimator.get)
print(min_key)

68


In [41]:
rf_model_on_full_data = RandomForestRegressor(n_estimators=68, max_leaf_nodes=4)
rf_model_on_full_data.fit(train_X, train_y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=4,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=68, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [50]:
test_data = pd.read_csv("test.csv")
test_X = test_data[features]
print(test_X.isna().sum())
test_X.fillna(0, inplace=True)


sc = StandardScaler()
sc.fit(test_X)
test_X = sc.transform(test_X)

#dummies = pd.get_dummies(test_X.Neighborhood)
#df_dummies = pd.concat([test_X, dummies], axis = 'columns')
#df_dummies.drop(['Neighborhood'], axis='columns', inplace=True)
#test_X = df_dummies


test_preds = rf_model_on_full_data.predict(test_X)
output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': test_preds})
output.to_csv('submission3.csv', index=False)




LotArea        0
MiscVal        0
2ndFlrSF       0
PoolArea       0
BsmtFinSF2     1
GrLivArea      0
TotalBsmtSF    1
GarageCars     1
dtype: int64


C:\Anaconda\lib\site-packages\pandas\core\frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
